In [1]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jan 13 10:30:20 2021

@author: Group 7
"""

# Install the different libraries
# !pip install dash
# !pip install dash-bootstrap-components
# !pip install dash-core-components
# !pip install dash-html-components
# !pip install pandas

# Import the libraries
from jupyter_dash
import datetime
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.graph_objs as go
from dash.exceptions import PreventUpdate
from dash.dependencies import Input, Output, State, ALL
import g7_functions as g7

In [2]:
app = JupyterDash(external_stylesheets=[dbc.themes.BOOTSTRAP],
                suppress_callback_exceptions=True)


# Data import
df_articles = pd.read_json('./data/df_clean_tagNA.json')
df_news = pd.read_json('./data/Pred_Global_V0.json')
df_resume = pd.read_json('./data/summary_v1.json')
df_stories = pd.read_json('./data/stories_v0.json')
df_gamme_ges = pd.read_json('./data/Res_gestion_semi_supervised_V1.json')


# Remove Nan value
df_articles.dropna(subset=['art_title', 'art_url'])
# df_articles['art_title']
df_articles = df_articles.reset_index(drop=True)


# Clean the date with the function 'clean_date' and replace the nan by a date .
df_articles['art_date'] = \
    df_articles['art_extract_datetime'].apply(g7.clean_date)
df_articles['art_date'] = df_articles['art_date'].fillna('04/01/2021')

# Convert colums art_id in string
df_news['art_id'] = df_news['art_id'].astype('str')
df_resume['art_id'] = df_resume['art_id'].astype('str')
df_gamme_ges['art_id'] = df_gamme_ges['art_id'].astype('str')
df_articles['art_id'] = df_articles['art_id'].astype('str')


df_final = df_articles


# Retrieves data equal to 1 in the df
df_gamme_ges = df_gamme_ges[df_gamme_ges['gamme_gestion'] == 1]
df_news = df_news[df_news['Innovant'] == 1]


df_final = df_final.merge(df_news, on='art_id')
df_final = df_final.merge(df_gamme_ges, on='art_id')
df_final = df_final.merge(df_resume, on='art_id')
df_final = df_final.drop([
        'Innovant',
        'gamme_gestion',
        'art_content_html',
        'art_extract_datetime',
        'src_type',
        'art_lang'
        ], axis=1)


list_ind_drop = df_final[df_final['art_title'] == ' '].index.values.astype(int)

df_final = df_final.drop(list_ind_drop)

df_final = df_final.reset_index(drop=True)


# Retrieves the list of the unique years in the df
liste_annee = g7.year_list(df_final)
df_final['art_date'] = pd.to_datetime(df_final['art_date'], format='%d/%m/%Y')


liste_source = df_final['src_name'].value_counts()[:20].index


# Treatment of the tags
tags = g7.theme_treatment(df_final)
dict_tags = {}
for i in df_final['art_tag']:
    if i is None:
        continue
    else:
        for j in tags:
            if j in i:
                if j in dict_tags:
                    dict_tags[j] += 1
                else:
                    dict_tags[j] = 1


# Retrieves the 10 top tags
top_tags = sorted(dict_tags.items(), key=lambda t: t[1])[-11:-1]


tops_tags_index = []
for i in range(len(top_tags)):
    tops_tags_index.append(top_tags[i][0])


# Replace the None img in the dataframe by the logo of SID
list_img = []
for i in df_final['src_img']:
    if i is None or '.gif' in i:
        list_img.append('./static/images/logo_sid.jpg')
    else:
        list_img.append(i)


df_final['src_img'] = list_img


tags = g7.theme_treatment(df_final)


# Order the article for have the most recent in first on the dash
df = df_final.sort_values(by=['Nouveau', 'art_date'], ascending=[0, 0])


# Retrieves the theme list
themes = df['Theme'].unique()
liste_theme = []
for theme in themes:
    if theme not in liste_theme:
        liste_theme.append(theme)


dict_tags = {}
for i in df['art_tag']:
    if i is None:
        continue
    else:
        for j in tags:
            if j in i:
                if j in dict_tags:
                    dict_tags[j] += 1
                else:
                    dict_tags[j] = 1

top_tags = sorted(dict_tags.items(), key=lambda t: t[1])[-11:-1]

tags_index = 11

NameError: name 'JupyterDash' is not defined

In [ ]:
# Build the search bar with a submit button
search_bar = dbc.Row([
        dbc.Col(
            dbc.Input(
                id='input_search',
                type='search',
                placeholder='Rechercher un mot clé',
                debounce=True
                )
            ),
        dbc.Col(
            dbc.Button(
                id='submit_button',
                color='primary',
                children='Rechercher',
                href='/'
                ), width='auto'
            ),
        ],
                     no_gutters=True,
                     id='search_bar',
                     )


# Build the navbar with the logo of Berger levrault, the different pages links
# and the search bar
nav_bar = dbc.Navbar([
        html.A(
            # Use row and col to control vertical
            # alignment of logo / brand
            dbc.Row([
                dbc.Col(
                    dbc.CardImg(
                        src='./static/images/logo_bl_short.jpg',
                        alt='',
                        top=True
                        ),
                    id='nav_bar_logo'
                    )
                ],
                    align='center',
                    no_gutters=True,
                    ),
            href='/',
            ),
        dbc.NavLink(
            'Article',
            id='active_home_page',
            active=True,
            href='/',
            className='nav_tab_link'
            ),
        dbc.NavLink(
            'Statistiques',
            href='/stats',
            id='active_stat_tab',
            className='nav_tab_link'
            ),
        dbc.NavLink(
            'Stories',
            href='/stories',
            id='active_stor_tab',
            className='nav_tab_link'
            ),
        dbc.NavbarToggler(
            id='navbar_toggler',
            className='nav_tab_link'
            ),
        dbc.Collapse(
            search_bar,
            id='navbar_collapse',
            navbar=True
            )
        ],
                     id='nav_bar',
                     sticky='top'
                     )


# Build the side bar with the different filters : the numbers of article per
# page, the different themes in a drop down and a button for apply the filters
# and a button  for refresh filters
side_bar = html.Div([
        html.H2('Filtres'),
        html.Hr(),
        html.Div([
            dbc.Col([
                dbc.Col(
                    html.Div([
                        html.Label(
                            'Nombre d\'article par page'
                            ),
                        dcc.Input(
                            id='nb_article_page',
                            type='number',
                            value=10,
                            min=1,
                            max=len(df['art_title']),
                        )
                    ])
                ),
                dbc.Col(
                    html.Div([
                        html.Label(
                            'Thème'
                        ),
                        dcc.Dropdown(
                            id='themes_filters',
                            options=[
                                {'label': row,
                                 'value': row}
                                for row in liste_theme
                            ],
                            multi=True
                        )
                    ]), width='auto'
                ),
                html.Hr(),
                dbc.Col(
                    html.Div([
                        html.Button(
                            'Appliquer Filtres',
                            id='research_filt'
                        )
                        ])
                ),
                html.Hr(),
                dbc.Col(
                    html.Div([
                        html.Label(
                            'Filtres'
                        ),
                        html.Br(),
                        html.Button(
                            'Rafraichir Filtres',
                            id='reset_button'
                        )
                    ])
                ),
            ]),
        ]),
    ],
                    id='side_bar'
                    )


# Build also the side bar for the statistics page (for the first graph)
side_bar_stats = html.Div([
        html.H2('Stats'),
        html.Hr(),
        html.Div([
            dbc.Col([
                dbc.Col(
                    html.Div([
                        dbc.Label(
                            'Année'
                        ),
                        dcc.Dropdown(
                            id='annee_var',
                            options=[
                                {'label': row,
                                 'value': row}
                                for row in liste_annee
                            ],
                            value='2020',
                        ),
                    ])
                ),
                dbc.Col(
                    html.Div([
                        dbc.Label(
                            'Thème'
                        ),
                        dcc.Dropdown(
                            id='theme_var',
                            options=[
                                {'label': row,
                                 'value': row}
                                for row in liste_theme
                            ],
                        ),
                    ]),
                    width='auto'
                ),
                dbc.Col(
                    html.Div([
                        dbc.Label(
                            'Source'
                        ),
                        dcc.Dropdown(
                            id='source_var',
                            options=[
                                {'label': row,
                                 'value': row}
                                for row in liste_source
                            ],
                            optionHeight=75,
                        ),
                    ])
                ),
                html.Hr(),
                dbc.Col(
                    html.Div([
                        html.Label(
                            'Filtres'
                        ),
                        html.Br(),
                        html.Button(
                            'Rafraichir les filtres',
                            id='reset_button'
                        )
                    ])
                )
            ]),

        ]),

    ])


# Build the side bar for the statistics page (for the second graph)
sidebar2_stats = html.Div([
        html.Hr(),
        html.Div([
            dbc.Col([
                dbc.Col(
                    html.Div([
                        dbc.Label(
                            'Source'
                        ),
                        dcc.Dropdown(
                            id='source2_var',
                            options=[
                                {'label': row,
                                 'value': row}
                                for row in liste_source
                            ],
                            value=liste_source[0],
                            optionHeight=75,
                        ),
                    ])
                ),
                html.Hr(),
                dbc.Col(
                    html.Div([
                        html.Label(
                            'Filtres'
                        ),
                        html.Br(),
                        html.Button(
                            'Rafraichir les filtres',
                            id='reset2_button'
                        )
                    ])
                )
            ]),
        ]),
    ])


# Build a nav bar for choose the number of the page
nb_pages = 10

pages = dbc.NavbarSimple(
        children=[
            dbc.Row([
                dbc.Col(
                    html.H5(
                        'Page n°',
                        id='page_number'
                    )
                ),
                dbc.Col(
                    html.Div([
                        dcc.Input(
                            id='num_page',
                            type='number',
                            value=1,
                            min=1
                        )
                    ])
                )
            ])
        ],
        color='#372E95',
        style={'height': '8px auto'}
)


# Build the tags content: we build card with a title, the number of article of
# the tag and a button who allows to show the articles of the tag.
def render_tag():
    """Documentation

    Return a list of tag

    Out:
        list_stories: List of tags
    """
    list_tags = []
    for i in range(len(top_tags)-1, -1, -1):
        list_tags.append(
            html.Div([
                dbc.Col(
                    html.Div([
                        dbc.Card([
                            dbc.CardBody([
                                html.H5(
                                    top_tags[i][0].capitalize(),
                                    className='card_title'
                                ),
                                html.H6(
                                    'Nombre d\'article : ' +
                                    str(top_tags[i][1]),
                                    className='card_subtitle'
                                ),
                                dbc.Button(
                                    'Afficher les articles du tag',
                                    id={
                                        'type': 'tags_button',
                                        'index': i},
                                    className='button'
                                ),
                            ])
                        ],
                                 id='render_tag'
                                 ),
                    ],
                             id='div_tag'
                             )
                )
            ])
        )
    return list_tags


# Build the stories content, also with a title, the source and the date of
# the storie. Cardlink allows to put a link on the title for show the articles
def render_stories() -> list:
    """Documentation

    Return a list of stories

    Out:
        list_stories: List of stories
    """
    list_stories = []
    card = []
    stories_2 = df_stories.sort_values(by='story_pos')
    stories_pos = stories_2['story_id'].unique()
    for j in stories_pos:
        for i in range(len(stories_2)):
            if stories_2['story_id'][i] == j:
                if stories_2['story_pos'][i] == 0:
                    card.append(
                        html.Div([
                            dbc.CardLink(
                                html.H4(
                                    df['art_title'][df['art_id'] ==
                                                    str(stories_2['art_id'][i])],
                                    className='card_title'
                                ),
                                href='/page-' +
                                str(int(df['art_id'][df['art_id'] ==
                                                     str(stories_2['art_id'][i])].values)) +
                                '-s1'
                            ),
                            html.P(
                                'Source : ' +
                                df['src_name'][df['art_id'] ==
                                               str(stories_2['art_id'][i])]
                            ),
                            html.P(
                                df['art_date'][df['art_id'] ==
                                               str(stories_2['art_id'][i])]
                            )
                        ])
                    )
                    card.append(html.Hr())
                else:
                    card.append(
                        html.Div([
                            dbc.CardLink(
                                html.H6(
                                    df['art_title'][df['art_id'] ==
                                                    str(stories_2['art_id'][i])]
                                ),
                                href='/page-'
                                +str(int(df['art_id'][df['art_id'] ==
                                                      str(stories_2['art_id'][i])].values))
                                +'-s2'
                            ),
                            html.P(
                                'Source : ' +
                                df['src_name'][df['art_id'] ==
                                               str(stories_2['art_id'][i])]
                            ),
                            html.P(
                                df['art_date'][df['art_id'] ==
                                               str(stories_2['art_id'][i])]
                            )
                        ])
                    )
        list_stories.append(
            dbc.Card([
                dbc.CardBody(
                    card
                )
            ],
                     className='w-100 mb-3'
                     )
        )
        card = []
    return list_stories


# Define the different div for print differents contents
card = html.Div(id='out_card')
article = html.Div(id='out_article')
show_tag = html.Div(render_tag())
show_stories = html.Div(render_stories())


# This part is the layout of the app. He describes what the dashboard
# look likes.
app.layout = html.Div([
        nav_bar,
        html.Hr(),
        dcc.Location(
            id='url',
            refresh=False
        ),
        html.Div(
            id='page_content'
        ),
        html.Div([
            side_bar
        ],
                 style={'display': 'none'}
                 )
    ])

# This part is the layout for the Home page. It's the page who show all
# articles
index_p_layout = [dbc.Row([
        dbc.Col([
            side_bar
        ],
                width='100%'
                ),
        dbc.Col([
            html.Div([
                html.Label(
                    'Période :  '
                ),
                dcc.DatePickerRange(
                    id='date_picker_range',
                    className='ssr-dash-core-components-datepickerrange',
                    start_date=min(df['art_date']),
                    end_date=datetime.date.today(),
                    calendar_orientation='vertical',
                    display_format='DD/MM/YYYY'
                )
            ],
                     style={
                         'text-align': 'center',
                         'margin-bottom': '10px',
                         'margin-right': '15px'
                         }
                     ),
            card
        ]),
        dbc.Col(
            show_tag,
            width='30%'
        )
        ],
                          no_gutters=True,
                          ),
                  pages
                  ]

# This part is the layout for the statistique page
stat_p_layout = [
        dbc.Row([
            dbc.Col([
                side_bar_stats
            ],
                    width='100%'
                    ),
            dbc.Col(
                html.Div(
                    dcc.Graph(
                        id='cluster_graph'
                    ),
                )
            )
        ],
                no_gutters=True
                ),
        dbc.Row([
            dbc.Col([
                sidebar2_stats
            ],
                    width='100%'
                    ),
            dbc.Col(
                html.Div(
                    dcc.Graph(
                        id='source_graph'
                    ),
                )
            )
        ],
                no_gutters=True
                )]


# Article layout for when we show just a article in particular.
article_p_layout = dbc.Col(article)

# The stories page layout.
story_p_layout = dbc.Col(show_stories)


# Callback. The callback is for the interactivity of the dash

# add callback for toggling the collapse on small screens
@app.callback(
    Output('navbar_collapse', 'is_open'),
    [Input('navbar_toggler', 'n_clicks')],
    [State('navbar_collapse', 'is_open')])
def toggle_collapse(n: int, is_open: bool) -> bool:
    """Documentation

    Parameters:
        n: Collapse button
        is_open: bool

    Out:
        is_open : bool
    """
    if n:
        return not is_open
    return is_open


@app.callback(
    Output('themes_filters', 'value'),
    Output('date_picker_range', 'start_date'),
    Output('date_picker_range', 'end_date'),
    Output('nb_article_page', 'value'),
    Output('input_search', 'value'),
    Output('num_page', 'value'),
    Input('reset_button', 'n_clicks'),
    Input('active_home_page', 'n_clicks'))
def reset_filter(n_click: int, n_click_2: int):
    """Documentation

    Reset article filters

    Parameters:
        n_click: Button to refresh the filters
        n_click_2: Button to position yourself in "Article"

    Raises:
        PreventUpdate DESCRIPTION: Prevent from updating the reminder output

    Out:
        themes_filters: None
        start_date: The earliest date of publication
        end_date: Today's date
        nb_article_page: Number of articles to display
        input_search: None
        num_page: Number of page

    """
    global tags_index

    if n_click is None and n_click_2 is None:
        raise PreventUpdate

    tags_index = 11
    return None, min(df['art_date']), datetime.date.today(), 10, None, 1


@app.callback(
    Output('annee_var', 'value'),
    Output('theme_var', 'value'),
    Output('source_var', 'value'),
    Input('reset_button', 'n_clicks'))
def reset_filt_stat(btn_rest):
    """Documention

    Reset the first stats graph

    Parameters:
        n: Stats filter reset button

    Out:
        annee_var: default 2020
        theme_var: None
        source_var: None
    """

    return 2020, None, None


@app.callback(
    Output('source2_var', 'value'),
    Input('reset2_button', 'n_clicks'))
def rst_filt_stat2(n):
    """Documention

    Reset the second stats graph

    Parameters:
        n: Stats filter reset button for the second graphique

    Out:
        source2_var: None
    """

    return 'Vie publique'


@app.callback(
    Output('out_card', 'children'),
    Output('num_page', 'max'),
    [Input('active_home_page', 'n_clicks')],
    [Input('research_filt', 'n_clicks')],
    [Input('num_page', 'value')],
    [Input({'type': 'tags_button', 'index': ALL}, 'n_clicks')],
    [Input('submit_button', 'n_clicks')],
    [Input('input_search', 'value')],
    [State('themes_filters', 'value')],
    [State('nb_article_page', 'value')],
    [State('date_picker_range', 'start_date')],
    [State('date_picker_range', 'end_date')],
    [State('input_search', 'value')])
def update_card_div(n, n_1, num_page, n_2, clicks, event, theme, number,
                    start_date, end_date, input_value):
    """Documention

    Display of articles

    Parameters:
        n: "Article" tab
        n_1: Filter application button
        num_page: Number of page
        n_2; Button for show list of articles for a tag
        clicks: Button for the research
        event: Search bar
        theme: Filter for themes
        number: Filter for number
        start_date: Filter min date
        end date: Filter for max date
        input_value : Sear_bar

    Out:
        out_card: Display
        num_page: Number of page
    """

    global tags_index

    if len(start_date) > 10:
        start_date = start_date[:10]

    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]

    if 'active_home_page' in changed_id:
        event = None
        num_page = 1
        tags_index = 11
        theme = None
        number = 10
        start_date = str(min(df['art_date']))[:10]
        end_date = datetime.date.today()

    filtered_df = df
    if 'research_filt' in changed_id:
        filtered_df = df
        if theme is not None:
            filtered_df = filtered_df[filtered_df['Theme'].isin(theme)]

        filtered_df = filtered_df.reset_index(drop=True)

        if start_date is not None and end_date is not None:
            filtered_df = filtered_df[(filtered_df['art_date'] >= start_date) & (filtered_df['art_date'] <= end_date)]

        filtered_df = filtered_df.reset_index(drop=True)

    filtered_df['art_tag'] = filtered_df['art_tag'].astype('str')

    if 'index' in changed_id:
        filtered_df = df
        tags_index = int(changed_id[9])
        filtered_df = filtered_df[filtered_df['art_tag'].str.contains(top_tags[tags_index][0], case=False)]
    elif tags_index != 11:
        filtered_df = df
        filtered_df = filtered_df[filtered_df['art_tag'].str.contains(top_tags[tags_index][0], case=False)]

    filtered_df = filtered_df.reset_index(drop=True)

    element = []
    if 'submit_button' in changed_id or event is not None:
        filtered_df = df
        for nb in range(len(filtered_df)):
            t = input_value
            if t.lower() in filtered_df['art_title'][nb].lower() or t.lower() in filtered_df['art_summary300'][nb].lower() or t.lower() in filtered_df['art_tag'][nb].lower():
                element.append(filtered_df['art_id'][nb])

        if element != []:
            filtered_df = filtered_df[filtered_df['art_id'].isin(element)]

        filtered_df = filtered_df.reset_index(drop=True)

    length_df = len(filtered_df['art_title'])

    if num_page*number > length_df:
        end = length_df
        filtered_df = filtered_df[((num_page*number)-number):length_df]
    else:
        filtered_df = filtered_df[((num_page*number)-number):((num_page*number))]
        end = num_page*number

    cards = []

    if (tags_index != 11):
        cards = [html.H5(top_tags[tags_index][0]), html.Hr()]

    for i in range(((num_page*number)-number), end):
        cards.append(
            html.Div([
                dbc.Col(
                    html.Div([
                        dbc.Card([
                            dbc.CardBody([
                                html.Div(
                                    dbc.CardImg(
                                        src=filtered_df['src_img'][i],
                                        alt='',
                                        top=True
                                    ),
                                    className='card_img_home'
                                ),
                                html.Hr(),
                                dbc.CardLink(
                                    html.H5(
                                        html.A(
                                            filtered_df['art_title'][i],
                                            href='/page-' +
                                            str(filtered_df['art_id'][i]),
                                            className='card_title'
                                        )
                                    )
                                ),
                                html.P(
                                    filtered_df['art_summary50'][i]
                                ),
                                html.Hr(),
                                html.Div([
                                    html.P([
                                        html.B('Source : '),
                                        filtered_df['src_name'][i].capitalize()
                                    ],
                                           style={'float': 'left'}
                                           ),
                                    html.P([
                                        html.B('Date : '),
                                        str(filtered_df['art_date'][i])[:10]
                                    ],
                                           style={'text-align': 'right'}
                                           )
                                ],
                                         style={'clear': 'both'}
                                         )
                            ])
                        ],
                                 className='w-100 mb-3'
                                 )
                    ])
                                )
                ])
            )

    cards += [html.Div([
        dbc.Button(
            'Retour en haut',
            className='button',
            href='#'
        )
    ])]
    if cards == []:
        cards.append(html.H5('Aucun article pour cette recherche'))
    return cards, (length_df//number)+1


# Content article
@app.callback(
    Output('out_article', 'children'),
    [Input('url', 'pathname')])
def toggle_article(pathname):
    """

    Detailed display of an article

    Parameters:
        pathname: Détailed article url link

    Raise:
        PreventUpdate DESCRIPTION: Prevent from updating the reminder output

    Out:
        out_article: Detailled display

    """

    if pathname == '/':
        raise PreventUpdate

    try:
        index = int(pathname.split('-')[1])
        df_article = df[df['art_id'] == str(index)]
        df_article = df_article.reset_index()

        if len(pathname.split('-')) == 2:
            article = html.Div([
                dbc.Col(html.Div([
                    dbc.Card([
                        dbc.CardBody([
                            dbc.Button(
                                'Retour à la page précédente',
                                className='button',
                                href='/'
                            ),
                            html.Hr(),
                            dbc.CardLink(
                                html.H5(
                                    df_article['art_title'][0],
                                    className='card_title'
                                )
                            ),
                            html.Hr(),
                            dbc.CardImg(
                                src=df_article['src_img'][0],
                                alt='',
                                top=True,
                                style={'width': '100%'}
                            ),
                            html.Br(),
                            html.Br(),
                            html.P(
                                df_article['art_summary300'][0],
                                style={'text-align': 'justify'}
                            ),
                            html.Br(),
                            html.Hr(),
                            html.Div([
                                html.P([
                                    html.B('Source : '),
                                    html.A(
                                        df_article['src_name'][0],
                                        href=df_article['art_url'][0],
                                        target='blank',
                                        id='source_article'
                                    )
                                ], style={'float': 'left'}),
                                html.P([
                                    html.B('Date : '),
                                    df_article['art_date'][0]
                                    ],
                                       style={'text-align': 'right'}
                                       )
                                ],
                                     style={'clear': 'both'}
                                     ),
                            html.Div([
                                html.P([
                                    html.B('Tag(s) : '),
                                    ' '.join(df_article['art_tag'][0])
                                ],
                                       style={'text-align': 'right'}
                                       )
                            ], style={'clear': 'both'}
                                    ),
                            html.Hr(),
                            html.Div(
                                dbc.Button(
                                    'Retour en haut',
                                    className='button',
                                    href='#'
                                ),
                                style={'text-align': 'center'}
                                ),
                        ])],
                             className='w-100 mb-3'
                             ),
                                ],
                                 style={
                                     'width': 'auto',
                                     'display': 'flex',
                                     'justify-content': 'center'
                                     }
                                )
                        )])
        elif len(pathname.split('-')) == 3:
            article = html.Div([
                dbc.Col(html.Div([
                    dbc.Card([
                        dbc.CardBody([
                            dbc.Button(
                                'Retour à la page précédente',
                                className='button',
                                href='/stories'
                            ),
                            html.Hr(),
                            dbc.CardLink(
                                html.H5(
                                    df_article['art_title'][0],
                                    className='card_title',
                                    style={
                                        'width': 'auto',
                                        'align': 'justify',
                                        'text-align': 'justify'
                                        }
                                    )
                                ),
                            html.Hr(),
                            dbc.CardImg(
                                src=df_article['src_img'][0],
                                alt='',
                                top=True,
                                style={'width': '100%'}
                                ),
                            html.Br(),
                            html.Br(),
                            html.P(
                                df_article['art_summary300'][0],
                                style={'text-align': 'justify'}
                                ),
                            html.Br(),
                            html.Hr(),
                            html.Div([
                                html.P([
                                    html.B('Source : '),
                                    html.A(
                                        df_article['src_name'][0],
                                        href=df_article['art_url'][0],
                                        target='blank',
                                        id='source_article'
                                    )
                                ],
                                       style={'float': 'left'}
                                       ),
                                html.P([
                                    html.B('Date : '),
                                    df_article['art_date'][0]
                                    ],
                                       style={'text-align': 'right'}
                                       )
                            ],
                                     style={'clear': 'both'}
                                     ),
                            html.Div([
                                html.P([
                                    html.B('Tag(s) : '),
                                    df_article['art_tag'][0]
                                    ],
                                       style={'text-align': 'right'}
                                       )
                                ],
                                     style={'clear': 'both'}),
                            html.Hr(),
                            html.Div(
                                dbc.Button(
                                    'Retour en haut',
                                    className='button',
                                    href='#'
                                    ),
                                style={'text-align': 'center'}
                                ),
                            ])
                        ],
                             className='w-100 mb-3'
                             ),
                    ],
                                 style={
                                     'width': 'auto',
                                     'display': 'flex',
                                     'justify-content': 'center'
                                     }
                                ))])
        return article
    except:
        pass


# Callback for update data in graphs
@app.callback(
    Output('cluster_graph', 'figure'),
    [
        Input('annee_var', 'value'),
        Input('theme_var', 'value'),
        Input('source_var', 'value'),
    ])
def nb_articles(year, theme, source) -> go:
    """

    Update data for the first graphs

    Parameters:
        year: Year of Article
        theme: Theme of Article
        source: Source of article

    Out:
        cluster_graph: Graphic

    """
    articles_months = [0]*12
    articles_2 = df

    if theme is not None:
        articles_2 = articles_2[articles_2['Theme'] == theme]

    if source is not None:
        articles_2 = articles_2[articles_2['src_name'] == source]

    articles_2 = articles_2.reset_index(drop=True)

    for i in range(1, len(articles_2['art_date'])):
        month = int(articles_2['art_date'][i].strftime('%m'))
        year2 = int(articles_2['art_date'][i].strftime('%Y'))
        if int(year2) == int(year):
            articles_months[month-1] += 1

    fig = go.Figure(
        data=[
            go.Scatter(
                x=['jan',
                   'feb',
                   'mar',
                   'apr',
                   'may',
                   'jun',
                   'jul',
                   'aug',
                   'sep',
                   'oct',
                   'nov',
                   'dec'],
                y=articles_months)],
        layout=go.Layout(
            title=go.layout.Title(
                text='Nombre d\'articles publié par mois en  '+str(year)
                        )
            )
        )
    return (fig)


# Callback for update data in graphs
@app.callback(
    Output('source_graph', 'figure'),
    Input('source2_var', 'value'),
)
def tags_source(source):
    """
    Update data for the second graphs

    Parameters:
        source: Source of article

    Out:
        source_graph: Graphic

    """
    number_source = [0]*10
    articles_3 = df

    articles_3 = articles_3[articles_3['src_name'] == source]

    articles_3 = articles_3.reset_index(drop=True)

    articles_3['art_tag'] = articles_3['art_tag'].astype('str')

    for i in range(0, len(articles_3)):
        for j in range(0, len(tops_tags_index)):
            if tops_tags_index[j] in articles_3['art_tag'][i]:
                number_source[j] += 1

    fig = go.Figure(
        data=[
            go.Scatter(
                x=tops_tags_index,
                y=number_source)],
        layout=go.Layout(
            title=go.layout.Title(
                text='Nombre d\'articles par tags pour  '+str(source))))

    return (fig)


@app.callback(
    Output('active_home_page', 'active'),
    Output('active_stat_tab', 'active'),
    Output('active_stor_tab', 'active'),
    [Input('url', 'pathname')])
def toggle_active_links(pathname):
    """Documentation
    Show the active tab

    Parameters:
        pathname: Détailed article url link

    Out:

    """
    if pathname == '/' or 'page' in pathname:
        return True, False, False
    elif pathname == '/stats':
        return False, True, False
    else:
        return False, False, True


# Content
@app.callback(Output('page_content', 'children'),
              [Input('url', 'pathname')])
def render_page_content(pathname):
    """Documentation
    Show content according to the url

    Parameters:
        pathname: Détailed article url link

    Out:
        page_content: Layout for the associated url
    """
    if pathname == '/':
        return index_p_layout
    elif pathname == '/stats':
        return stat_p_layout
    elif pathname == '/stories':
        return story_p_layout
    else:
        return article_p_layout
    # If the user tries to reach a different page, return a 404 message
    return dbc.Jumbotron(
        [
            html.H1('404: Not found', className='text-danger'),
            html.Hr(),
            html.P(f'The pathname {pathname} was not recognised...'),
        ]
    )


app.title = 'Berger Levrault News'

if __name__ == '__main__':
    app.run_server(debug=False, port=8054)
